# Config

In [1]:
!nvidia-smi

Mon Jun  9 13:01:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import os

from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    pipeline,
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

In [10]:
if not torch.cuda.is_available():
    print("Cuda is not available. Exiting.")

In [11]:
BASE_MODEL_ID = "EleutherAI/gpt-neo-125M" # TODO: check few different models
LORA_MODEL_OUTPUT_DIR = "./hate-speech-lora-model"
TRAIN_FILE = "/content/hate_train.csv"
TEST_FILE = "/content/hate_test_data.txt"
PREDICTION_FILE = "pred.csv"
DO_DATA_AUGMENTATION = True


SEED = 42 # reproductivity

# if os.path.exists(TEST_FILE):
#     print(f"Loading training data from {TRAIN_FILE}...")
# else:
#     raise FileNotFoundError(f"Training file {TRAIN_FILE} not found.")


## Labels, Tokenizer etc.

In [12]:
id2label = {0: "no-hate", 1: "hate"}
label2id = {"no-hate": 0, "hate": 1}
NUM_LABELS = len(id2label)

In [13]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
PROMPT_TEMPLATE_FINETUNE = (
    "Classify the following text as 'hate' or 'no-hate'.\n\n"
    "Text: {text}\n"
    "Label: {label_str}{eos_token}"
)

PROMPT_TEMPLATE_INFERENCE = (
    "Classify the following text as 'hate' or 'no-hate'.\n\n"
    "Text: {text}\n"
    "Label:"
)

# TODO prompta chyba lepiej po polsku dla polskich modeli? ale idk

## Dataloading

In [22]:
import pandas as pd
try:
    df_train_full = pd.read_csv(TRAIN_FILE)
    with open(TEST_FILE, 'r', encoding='utf-8') as f:
        test_texts = [line.strip() for line in f]
    df_test = pd.DataFrame(test_texts, columns=['sentence'])
except FileNotFoundError as e:
    print(f"err: no file {e.filename}")
    exit()

print(f"Loaded {len(df_train_full)} training samples and  test samples.")

print(df_train_full.head())
print()
print(df_train_full['label'].value_counts())

Loaded 10041 training samples and  test samples.
                                            sentence  label
0  Dla mnie faworytem do tytułu będzie Cracovia. ...      0
1  @anonymized_account @anonymized_account Brawo ...      0
2  @anonymized_account @anonymized_account Super,...      0
3  @anonymized_account @anonymized_account Musi. ...      0
4    Odrzut natychmiastowy, kwaśna mina, mam problem      0

label
0    9190
1     851
Name: count, dtype: int64


### Balancing classes (augmentation)

In [23]:
print('Original distribution:')
print(df_train_full['label'].value_counts(normalize=True))

Original distribution:
label
0    0.915247
1    0.084753
Name: proportion, dtype: float64


In [24]:
df_majority = df_train_full[df_train_full['label'] == 0]
df_minority = df_train_full[df_train_full['label'] == 1]

df_minority_oversampled = df_minority.sample(
    n=len(df_majority),
    replace=True,
    random_state=SEED
)

In [25]:
df_train_balanced = pd.concat([df_majority, df_minority_oversampled])


df_train_balanced = df_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [26]:
print('Balanced distribution:')
print(df_train_balanced['label'].value_counts(normalize=True))
print(f"\nBalanced class nums \n{df_train_balanced['label'].value_counts().to_string()}")


Balanced distribution:
label
1    0.5
0    0.5
Name: proportion, dtype: float64

Balanced class nums 
label
1    9190
0    9190


In [27]:
#
df_train_full = df_train_balanced


### Data augmentation v2


In [ ]:
# TODO mozna zobaczyc czy to ma wiekszy sens zamiast tego powyzej^

# if DO_DATA_AUGMENTATION:
#     print("\n--- Step 1a: Augmenting Data (Back-Translation) ---")
#     print("This may take a few minutes...")
#     try:
#         # Initialize translation pipelines
#         translator_pl_en = pipeline("translation", model="Helsinki-NLP/opus-mt-pl-en", device=0 if torch.cuda.is_available() else -1)
#         translator_en_pl = pipeline("translation", model="Helsinki-NLP/opus-mt-en-pl", device=0 if torch.cuda.is_available() else -1)

#         def back_translate(text):
#             try:
#                 en_text = translator_pl_en(text, max_length=128)[0]['translation_text']
#                 pl_text_augmented = translator_en_pl(en_text, max_length=128)[0]['translation_text']
#                 return pl_text_augmented
#             except Exception as e:
#                 print(f"Error during translation: {e}")
#                 return text  # Return original text on error

#         # Augment the minority class (hate speech) to balance the dataset
#         df_augmented_list = []
#         # We assume the 'hate' class (1) is the minority
#         texts_to_augment = df_train_full[df_train_full['label'] == 1]['text'].tolist()
#         print(f"Augmenting {len(texts_to_augment)} samples for the 'hate' class (label 1)")
#         for text in tqdm(texts_to_augment, desc="Augmenting 'hate' class"):
#             augmented_text = back_translate(text)
#             if augmented_text != text:
#                 df_augmented_list.append({'text': augmented_text, 'label': 1})

#         if df_augmented_list:
#             df_augmented = pd.DataFrame(df_augmented_list)
#             df_train_full = pd.concat([df_train_full, df_augmented], ignore_index=True)
#             print("\nTraining set after augmentation:")
#             print(f"New number of samples: {len(df_train_full)}")
#             print(df_train_full['label'].value_counts())

#     except Exception as e:
#         print(f"Could not perform data augmentation: {e}. Continuing without it.")




### Datasets

In [43]:
train_df, val_df = train_test_split(df_train_full, test_size=0.15, random_state=42, stratify=df_train_full['label'])

raw_datasets = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(val_df),
    'test': Dataset.from_pandas(df_test)
})

def format_dataset_for_finetuning(examples):
    texts = examples['sentence']
    labels_int = examples['label']
    formatted_prompts = []
    for text, label_int in zip(texts, labels_int):
        label_str = id2label[label_int]
        formatted_prompts.append(
            PROMPT_TEMPLATE_FINETUNE.format(
                text=text,
                label_str=label_str,
                eos_token=tokenizer.eos_token
            )
        )
    return {"formatted_prompt": formatted_prompts}

def set_labels(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

tokenized_datasets = {}
for split, data in raw_datasets.items():
    if split in ['train', 'validation']: # not on test
        formatted_data = data.map(format_dataset_for_finetuning, batched=True)

        tokenized_split = formatted_data.map(
            lambda examples: tokenizer(
                examples["formatted_prompt"],
                truncation=True,
                max_length=256, # TODO potencjalnie zwiekszyz (zerknac jaka jest srednia dlugosc tekstu w danych + dlugosc promtu)
                padding="max_length"
            ),
            batched=True,
            remove_columns=data.column_names + ["formatted_prompt"]
        )
        tokenized_datasets[split] = tokenized_split.map(set_labels, batched=True)

print("\nPróbka danych po tokenizacji:")
print(tokenized_datasets["train"][0])
print("\nZdekodowany tekst próbki:")
print(tokenizer.decode(tokenized_datasets["train"][0]['input_ids']))


Map:   0%|          | 0/15623 [00:00<?, ? examples/s]

Map:   0%|          | 0/15623 [00:00<?, ? examples/s]

Map:   0%|          | 0/15623 [00:00<?, ? examples/s]

Map:   0%|          | 0/2757 [00:00<?, ? examples/s]

Map:   0%|          | 0/2757 [00:00<?, ? examples/s]

Map:   0%|          | 0/2757 [00:00<?, ? examples/s]


Próbka danych po tokenizacji:
{'input_ids': [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50

## Config LoRA

In [44]:
# bq_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True,
# )
# # TODO mialem err z  BitsAndBytesConfig



model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    # quantization_config=bq_config,
    trust_remote_code=True,
    device_map="auto",
    load_in_8bit=False
)

model.config.use_cache = False
model.config.pretraining_tp = 1

In [45]:
import subprocess
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "c_proj", "c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 1,622,016 || all params: 126,820,608 || trainable%: 1.2790


## Training

In [46]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [55]:
training_args = TrainingArguments(
    output_dir=LORA_MODEL_OUTPUT_DIR+"_v2",
    num_train_epochs=3,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir=f"{LORA_MODEL_OUTPUT_DIR}/logs",
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,
    greater_is_better=False,
    report_to="wandb",
    fp16=True,
    run_name="lora_hate_speech_v2",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


<ipython-input-55-455a692c2c85>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()


Step,Training Loss,Validation Loss


In [ ]:
trainer.save_model(LORA_MODEL_OUTPUT_DIR)
tokenizer.save_pretrained(LORA_MODEL_OUTPUT_DIR)
print(f"\nSaved in {LORA_MODEL_OUTPUT_DIR}")

## Inference

In [ ]:
base_model_for_inference = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    # quantization_config=bq_config,
    trust_remote_code=True,
    device_map="auto"
)
inference_model = PeftModel.from_pretrained(base_model_for_inference, LORA_MODEL_OUTPUT_DIR)
inference_model.eval()

In [ ]:
def classify_hate_speech(text_to_classify, model, tokenizer, max_new_tokens=5):
    """Generuje predykcję dla pojedynczego tekstu."""
    prompt = PROMPT_TEMPLATE_INFERENCE.format(text=text_to_classify)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            temperature=0.1,
            do_sample=False # greedy
        )

    generated_ids = outputs[0, inputs.input_ids.shape[1]:]
    prediction_text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip().lower()

    if "hate-speech" in prediction_text:
        return label2id["hate-speech"]
    elif "offensive" in prediction_text:
        return label2id["offensive"]
    else:
        # Fallback: jeśli model nie wygenerował żadnej z oczekiwanych etykiet,
        # zwracamy najbezpieczniejszą/najczęstszą klasę.
        print(f"UNKNOWN '{prediction_text}'. Attaching to class (no-hate).")
        return label2id["no-hate"]


In [ ]:
predictions = []
for text in tqdm(test_texts, desc="Generate predictions"):
    predicted_label = classify_hate_speech(text, inference_model, tokenizer)
    predictions.append(predicted_label)


## Save predictions

In [ ]:
df_predictions = pd.DataFrame(predictions)
df_predictions.to_csv(PREDICTION_FILE, header=False, index=False)

print("Zakończono pomyślnie!")
print(f"Plik z predykcjami '{PREDICTION_FILE}' został utworzony.")
print("\nPróbka predykcji:")
print(df_predictions.head())